
# Sockets 101 — Client & Server (Theory + Code)

This notebook teaches **sockets**, **clients**, and **servers** step by step, then builds a tiny **HTTP** server and client — just like the low-level examples used in *Django for Everybody* before jumping into frameworks.

**What you'll do:**
1. Understand the TCP socket model (in plain English)
2. Run a minimal **echo server** (raw sockets)
3. Talk to it with a **client**
4. Build a tiny **HTTP server** on top of sockets
5. Query it with a **manual HTTP client** and with **urllib**
6. Practice exercises to solidify the concepts

> Tip: Run servers in one kernel/terminal, and clients in another (or stop/start between runs). Use `Ctrl+C` to stop a running server cell.



## 1) Mental model: sockets, client, server (theory)

- A **socket** is a two-ended pipe between two programs over the network.
  - One end is on the **server**, the other on the **client**.
- **Server** lifecycle (blocking version):
  1. `socket()` → create a TCP socket (`AF_INET`, `SOCK_STREAM`)

  2. `bind((HOST, PORT))` → attach to an address/port

  3. `listen(backlog)` → start listening queue

  4. `accept()` → wait; returns a new per-connection socket

  5. `recv()/sendall()` → exchange bytes

  6. close connection (loop back to step 4)
- **Client** lifecycle:
  1. `socket()`

  2. `connect((HOST, PORT))` → TCP handshake

  3. `sendall()` request → `recv()` response

  4. close

**HTTP** is just *structured text over TCP* (request + headers + blank line + optional body). Frameworks like **Django** sit on top of these layers.


## 2) Echo server (raw TCP) — run this cell to start the server

In [1]:

import socket

HOST = "127.0.0.1"  # loopback (localhost)
PORT = 9000         # change if busy

def run_echo_server(host=HOST, port=PORT):
    # Create a TCP socket
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_sock:
        # Avoid 'Address already in use' on quick restarts
        server_sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

        # Bind and listen
        server_sock.bind((host, port))
        server_sock.listen(5)
        print(f"Echo server listening on {host}:{port} (Ctrl+C to stop)")

        while True:
            conn, addr = server_sock.accept()
            with conn:
                print(f"Connected by {addr}")
                while True:
                    data = conn.recv(1024)
                    if not data:  # client closed
                        break
                    # Echo back the exact bytes we got
                    conn.sendall(data)
                print(f"Connection closed: {addr}")

# Uncomment to run the server in this cell (it will block):
# run_echo_server()
print("Ready: call run_echo_server() to start. Stop with Kernel Interrupt or Ctrl+C.")


Ready: call run_echo_server() to start. Stop with Kernel Interrupt or Ctrl+C.


## 3) Echo client — run this in a second session/after server is up

In [2]:

import socket

HOST = "127.0.0.1"
PORT = 9000

def talk_to_echo(host=HOST, port=PORT):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((host, port))
        print("Connected. Type a message (empty line to exit).")
        while True:
            msg = input("You> ")
            if not msg:
                break
            sock.sendall(msg.encode("utf-8"))
            data = sock.recv(1024)
            print("Server>", data.decode("utf-8", errors="replace"))
    print("Bye!")

# Uncomment to try:
# talk_to_echo()
print("Ready: call talk_to_echo() after the server is running.")


Ready: call talk_to_echo() after the server is running.



## 4) Tiny HTTP server (raw sockets)

This is a minimal HTTP/1.1 server:

- Parses only the first line of the request

- Always replies `200 OK` with HTML

- Uses headers: `Content-Type`, `Content-Length`, `Connection: close`

Run the function below, then open your browser at **http://127.0.0.1:9000/**


In [3]:

import socket
from datetime import datetime

HOST = "127.0.0.1"
PORT = 9000

def build_http_response(body: str, status="200 OK", headers=None):
    headers = headers or {}
    default_headers = {
        "Content-Type": "text/html; charset=utf-8",
        "Content-Length": str(len(body.encode("utf-8"))),
        "Connection": "close",
    }
    default_headers.update(headers)
    head_lines = "\r\n".join(f"{k}: {v}" for k, v in default_headers.items())
    return f"HTTP/1.1 {status}\r\n{head_lines}\r\n\r\n{body}".encode("utf-8")

def run_http_server(host=HOST, port=PORT):
    response_body = f"""
    <!doctype html>
    <html>
      <head><meta charset="utf-8"><title>Mini HTTP</title></head>
      <body style="font-family: system-ui; padding: 2rem;">
        <h1>It works! 🎉</h1>
        <p>Served by a raw-socket HTTP server at {datetime.utcnow().isoformat()}Z</p>
      </body>
    </html>
    """
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind((host, port))
        s.listen(5)
        print(f"HTTP server at http://{host}:{port}/  (Ctrl+C to stop)")
        while True:
            conn, addr = s.accept()
            with conn:
                request = conn.recv(4096).decode("utf-8", errors="replace")
                first_line = request.split("\r\n", 1)[0]
                print(f"[{addr}] {first_line}")
                conn.sendall(build_http_response(response_body))

# Uncomment to run:
# run_http_server()
print("Ready: call run_http_server() to start the HTTP server.")


Ready: call run_http_server() to start the HTTP server.


## 5) Manual HTTP client (raw sockets)

In [4]:

import socket

HOST = "127.0.0.1"
PORT = 9000

def manual_http_get(host=HOST, port=PORT, path="/"):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((host, port))
        req = (
            f"GET {path} HTTP/1.1\r\n"
            f"Host: {host}:{port}\r\n"
            "User-Agent: RawSocketDemo/1.0\r\n"
            "Connection: close\r\n"
            "\r\n"
        )
        sock.sendall(req.encode("utf-8"))
        chunks = []
        while True:
            data = sock.recv(4096)
            if not data:
                break
            chunks.append(data)
    resp = b"".join(chunks).decode("utf-8", errors="replace")
    print(resp)

# Uncomment after starting run_http_server():
# manual_http_get("/")
print("Ready: call manual_http_get('/') while the HTTP server is running.")


Ready: call manual_http_get('/') while the HTTP server is running.


## 6) Same idea with urllib (high-level)

In [5]:

from urllib.request import urlopen

def urllib_fetch(url="http://127.0.0.1:9000/"):
    with urlopen(url) as resp:
        print("Status:", resp.status)
        print("Headers:")
        for k, v in resp.getheaders():
            print(f"  {k}: {v}")
        print("\nBody:\n")
        print(resp.read().decode("utf-8", errors="replace"))

# Uncomment after starting run_http_server():
# urllib_fetch()
print("Ready: call urllib_fetch() while the HTTP server is running.")


Ready: call urllib_fetch() while the HTTP server is running.



## 7) Exercises (try these)

1. **Echo+ Uppercase** — Modify the echo server to uppercase each message before echoing.

2. **HTTP routing** — In `run_http_server`, change response based on `path` from the first request line:

   - `/` → current HTML

   - `/time` → JSON with `{"utc": "..."}` and `Content-Type: application/json`

3. **Manual client paths** — Call `manual_http_get("/time")` and verify JSON output.

4. **Port in use** — If you get `Address already in use`, set a different `PORT` (e.g., 9010) on both server and client.


When these feel easy, explore **threading** or **asyncio** to handle multiple clients at once.
